In [ ]:
import nrgpy
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
client_id = ""
client_secret = ""

In [ ]:
sites = nrgpy.cloud_sites(
    client_id=client_id,
    client_secret=client_secret,
)

In [ ]:
sites.sites_df

In [ ]:
exporter = nrgpy.cloud_export(
    client_id=client_id,
    client_secret=client_secret,
    site_number=110,
    start_date="2022-04-01",
    end_date="2022-05-15",
)

In [ ]:
exporter.export()

In [ ]:
reader = nrgpy.sympro_txt_read(filename=exporter.export_filepath)

In [ ]:
reader.ch_info

In [ ]:
ws_channels = reader.ch_info['Channel:'].loc[reader.ch_info['Units:'] == 'm/s']

In [ ]:
fig, ax = plt.subplots(
    figsize=(12, 5),
)

for ch in [c for c in ws_channels if int(c) < 20]:
    plot_col = [col for col in reader.data.columns if f"Ch{ch}_" in col and "avg" in col.lower()]
    ax.plot(
        reader.data['Timestamp'],
        reader.data[plot_col],
    )
    
plt.title(f"Wind speed plot for site {reader.site_description}")
plt.legend(labels=[f"Ch{c}" for c in ws_channels])

In [ ]:
reader.ch_info.loc[(reader.ch_info['Type:'] == "Anemometer") | (reader.ch_info['Type:'] == "Vane")]

In [ ]:
import numpy as np
from windrose import plot_windrose
import matplotlib.cm as cm

anem_col = [col for col in reader.data.columns if "Ch1_" in col and "avg" in col.lower()]
vane_col = [col for col in reader.data.columns if "Ch13_" in col and "avg" in col.lower()]

windrose_df = pd.DataFrame()
windrose_df['speed'] = reader.data[anem_col]
windrose_df['direction'] = reader.data[vane_col]

plot_windrose(windrose_df, kind='bar', bins=np.arange(0.01,8,1), cmap=cm.hot, lw=3)

plt.title(f"Wind rose diagram for {reader.site_description}")
plt.show()